In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from pymanticore.swift_analysis import HBTData, SOAPData
from stacking_backend import ClusterAnalysisPipeline, BasicPlotter, SummaryPlotter, MassScalingPlotter
from stacking_backend.data import load_pr4_data, PatchExtractor
from pymanticore.cosmology import ra_dec_to_galactic
import h5py
from pymanticore.analysis.matplotlib import get_mplstyle_path, ManticoreColors

# Initialize the analysis pipeline
pipeline = ClusterAnalysisPipeline()

# Create plotters
basic_plotter = BasicPlotter(pipeline.patch_extractor)
summary_plotter = SummaryPlotter()
mass_plotter = MassScalingPlotter()

mnras_style = "mnras"

In [ ]:
# Define examples and plotting function
def plot_example_patches(examples, basic_plotter, patch_size_deg=8.0, npix=128):
    """Plot patches for a list of example coordinates"""
    for i, example in enumerate(examples):
        print(f"\n📊 Example {i+1}: {example['title']}")
        
        fig, (ax1, ax2), patch_data, mask_patch = basic_plotter.plot_patch(
            ra=example['ra'],
            dec=example['dec'],
            patch_size_deg=patch_size_deg,
            npix=npix,
            title=example['title'],
            show_center=True,
            show_grid=True
        )
        
        print(f"Patch shape: {patch_data.shape}")

# Example coordinates to plot
examples = [
    {'ra': 194.95, 'dec': 27.98, 'title': 'Coma Cluster'},
    {'ra': 186.69, 'dec': 12.83, 'title': 'Virgo Cluster'},
    {'ra': 23.40, 'dec': 30.66, 'title': 'Perseus Cluster'},
    {'ra': 120.0, 'dec': 0.0, 'title': 'Random Position'},
    {'ra': 168.1, 'dec':  40.45, 'title': "test"}
]

In [ ]:
# Plot the example patches
plot_example_patches(examples, basic_plotter)

### Load MCXC cluster catalog

In [ ]:
# Load MCXC cluster catalog and plot 2 most massive
fname = "/cosma7/data/dp004/rttw52/Manticore/observational_data/mcxc_2_clusters/mcxc_clusters.hdf5"

with h5py.File(fname, "r") as f:
    _mcxc_data = f["all_clusters"][...]

_ra = np.degrees(_mcxc_data["phi"])
_dec = np.degrees(_mcxc_data["theta"])
gal_l, gal_b = ra_dec_to_galactic(_ra, _dec)
mcxc_data = {"gal_l": gal_l, "gal_b": gal_b, "z": _mcxc_data["z"], "m500": _mcxc_data["M500"]*1e14, "r500": _mcxc_data["R500"]}

# Find 2 most massive clusters and plot them
mass_indices = np.argsort(mcxc_data["m500"])[-2:][::-1]  # Get indices of 2 most massive, descending order

mcxc_massive_examples = []
for i, idx in enumerate(mass_indices):
    mcxc_massive_examples.append({
        'ra': _ra[idx],
        'dec': _dec[idx], 
        'title': f'MCXC Most Massive #{i+1} (M500={mcxc_data["m500"][idx]:.2e} M☉)'
    })

print(f"\nPlotting 2 most massive MCXC clusters:")
plot_example_patches(mcxc_massive_examples, basic_plotter)

### Load eROSITA cluster catalog

In [ ]:
# Load eROSITA cluster catalog and plot 2 most massive
fname = "/cosma7/data/dp004/rttw52/Manticore/observational_data/erosita_clusters/erosita_clusters.hdf5"

with h5py.File(fname, "r") as f:
    _eros_data = f["all_clusters"][...]

_ra = np.degrees(_eros_data["phi"])
_dec = np.degrees(_eros_data["theta"])
gal_l, gal_b = ra_dec_to_galactic(_ra,_dec)
eros_data = {"gal_l": gal_l, "gal_b": gal_b, "z": _eros_data["z"], "m500": _eros_data["M500"]*1e14, "r500": _eros_data["R500"]}

# Find 2 most massive clusters and plot them
mass_indices = np.argsort(eros_data["m500"])[-2:][::-1]  # Get indices of 2 most massive, descending order

eros_massive_examples = []
for i, idx in enumerate(mass_indices):
    eros_massive_examples.append({
        'ra': _ra[idx],
        'dec': _dec[idx],
        'title': f'eROSITA Most Massive #{i+1} (M500={eros_data["m500"][idx]:.2e} M☉)'
    })

print(f"\nPlotting 2 most massive eROSITA clusters:")
plot_example_patches(eros_massive_examples, basic_plotter)

## Load Manticore posterior clusters

In [ ]:
# Load Manticore posterior clusters and plot 2 most massive
import numpy as np
import h5py
from astropy.cosmology import Planck18, z_at_value
import astropy.units as u

fname = "/cosma7/data/dp004/rttw52/Manticore/new_analysis/clusters/temp/output/simplified_clusters.h5"

with h5py.File(fname, "r") as f:
    distances_mpc = f["clusters/median_properties/dist"][...]
    
    # Convert comoving distance to redshift using Planck18 cosmology
    redshifts = []
    for dist in distances_mpc:
        z = z_at_value(Planck18.comoving_distance, dist * u.Mpc)
        redshifts.append(z)
    
    manticore_data = {"gal_l": f["clusters/median_properties/gal_l"][...],
                      "gal_b": f["clusters/median_properties/gal_b"][...],
                      "m500": f["clusters/median_properties/SO_500_crit_TotalMass"][...],
                      "r500": f["clusters/median_properties/SO_500_crit_SORadius"][...],
                      "dist": distances_mpc,
                      "z": np.array(redshifts),
                      "ra": f["clusters/median_properties/ra"][...],
                      "dec": f["clusters/median_properties/dec"][...]}

# Find 2 most massive clusters and plot them
mass_indices = np.argsort(manticore_data["m500"])[-30:-20][::-1]  # Get indices of 2 most massive, descending order

manticore_massive_examples = []
for i, idx in enumerate(mass_indices):
    # Convert galactic to RA/Dec
    coord = SkyCoord(l=manticore_data["gal_l"][idx]*u.deg, b=manticore_data["gal_b"][idx]*u.deg, frame='galactic')
    icrs_coord = coord.icrs
    
    manticore_massive_examples.append({
        'ra': manticore_data["ra"][idx],
        'dec': manticore_data["dec"][idx],
        'title': f'Manticore Most Massive #{i+1} (M500={manticore_data["m500"][idx]:.2e} M☉)'
    })

print(f"\nPlotting 2 most massive Manticore clusters:")
plot_example_patches(manticore_massive_examples, basic_plotter)

In [ ]:
def plot_halo_overview(halo_data, z_min=None, z_max=None, figsize=(15, 10), cat_name=""):
    """
    Plot overview of cluster data.
    
    Parameters
    ----------
    halo_data : dict
        Dictionary containing cluster data with keys: 'gal_l', 'gal_b', 'z', 'm500', 'r500'
    z_min : float, optional
        Minimum redshift to include in plots
    z_max : float, optional
        Maximum redshift to include in plots
    figsize : tuple, optional
        Figure size (width, height)
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Apply redshift filter if specified
    if z_min is not None or z_max is not None:
        z_mask = np.ones(len(halo_data["z"]), dtype=bool)
        if z_min is not None:
            z_mask &= (halo_data["z"] >= z_min)
        if z_max is not None:
            z_mask &= (halo_data["z"] <= z_max)

        # Create filtered data
        filtered_data = {key: np.array(values)[z_mask] for key, values in halo_data.items()}
        title_suffix = f" (z: {z_min or 'min'} - {z_max or 'max'})"
    else:
        filtered_data = halo_data
        title_suffix = ""
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=figsize)
    fig.suptitle(f'{cat_name} Clusters Overview{title_suffix}', fontsize=16, fontweight='bold')

    # 1. Sky distribution in galactic coordinates
    ax1 = axes[0, 0]
    scatter = ax1.scatter(filtered_data["gal_l"], filtered_data["gal_b"], 
                         c=filtered_data["z"], cmap='viridis', alpha=0.7, s=20)
    ax1.set_xlabel('Galactic Longitude (deg)')
    ax1.set_ylabel('Galactic Latitude (deg)')
    ax1.set_title('Sky Distribution (colored by redshift)')
    ax1.set_xlim(0, 360)
    ax1.set_ylim(-90, 90)
    plt.colorbar(scatter, ax=ax1, label='Redshift')

    # 2. Redshift distribution
    ax2 = axes[0, 1]
    ax2.hist(filtered_data["z"], bins=30, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Redshift')
    ax2.set_ylabel('Number of Clusters')
    ax2.set_title('Redshift Distribution')
    ax2.axvline(np.median(filtered_data["z"]), color='red', linestyle='--', 
               label=f'Median: {np.median(filtered_data["z"]):.3f}')
    ax2.legend()

    # 3. Mass distribution
    ax3 = axes[0, 2]
    ax3.hist(np.log10(filtered_data["m500"]), bins=30, alpha=0.7, edgecolor='black')
    ax3.set_xlabel('log₁₀(M₅₀₀) [M☉]')
    ax3.set_ylabel('Number of Clusters')
    ax3.set_title('Mass Distribution')
    ax3.axvline(np.log10(np.median(filtered_data["m500"])), color='red', linestyle='--',
               label=f'Median: {np.median(filtered_data["m500"]):.1e} M☉')
    ax3.legend()

    # 4. R500 distribution
    ax4 = axes[1, 0]
    ax4.hist(filtered_data["r500"], bins=30, alpha=0.7, edgecolor='black')
    ax4.set_xlabel('R₅₀₀ [Mpc]')
    ax4.set_ylabel('Number of Clusters')
    ax4.set_title('R₅₀₀ Distribution')
    ax4.axvline(np.median(filtered_data["r500"]), color='red', linestyle='--',
               label=f'Median: {np.median(filtered_data["r500"]):.2f} Mpc')
    ax4.legend()

    # 5. Mass vs Redshift
    ax5 = axes[1, 1]
    scatter2 = ax5.scatter(filtered_data["z"], np.log10(filtered_data["m500"]), 
                          c=filtered_data["r500"], cmap='plasma', alpha=0.7, s=20)
    ax5.set_xlabel('Redshift')
    ax5.set_ylabel('log₁₀(M₅₀₀) [M☉]')
    ax5.set_title('Mass vs Redshift (colored by R₅₀₀)')
    plt.colorbar(scatter2, ax=ax5, label='R₅₀₀ [Mpc]')

    # 6. Data summary statistics
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    n_total = len(halo_data["z"])
    n_filtered = len(filtered_data["z"])
    
    stats_text = f"""Data Summary:
Total Clusters: {n_total:,}
Filtered Clusters: {n_filtered:,}
Filter: z ∈ [{z_min or 'min'}, {z_max or 'max'}]

Redshift:
  Range: {np.min(filtered_data["z"]):.3f} - {np.max(filtered_data["z"]):.3f}
  Mean: {np.mean(filtered_data["z"]):.3f}
  Std: {np.std(filtered_data["z"]):.3f}

Mass (M₅₀₀):
  Range: {np.min(filtered_data["m500"]):.1e} - {np.max(filtered_data["m500"]):.1e} M☉
  Mean: {np.mean(filtered_data["m500"]):.1e} M☉
  
R₅₀₀:
  Range: {np.min(filtered_data["r500"]):.2f} - {np.max(filtered_data["r500"]):.2f} Mpc
  Mean: {np.mean(filtered_data["r500"]):.2f} Mpc

Sky Coverage:
  Gal_l: {np.min(filtered_data["gal_l"]):.1f}° - {np.max(filtered_data["gal_l"]):.1f}°
  Gal_b: {np.min(filtered_data["gal_b"]):.1f}° - {np.max(filtered_data["gal_b"]):.1f}°"""

    ax6.text(0.05, 0.95, stats_text, transform=ax6.transAxes, fontsize=10,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

    plt.tight_layout()
    plt.show()

In [ ]:
# plot_mcxc_overview(mcxc_data)  # Plot all data
plot_halo_overview(manticore_data, z_min=0.001, z_max=0.1, cat_name="Manticore")

In [ ]:
plot_halo_overview(mcxc_data, z_min=0.001, z_max=0.1, cat_name="MCXC")

In [ ]:
plot_halo_overview(eros_data, z_min=0.001, z_max=0.1, cat_name="eROSITA")

In [ ]:
l_min = 20
l_max = 80

def generate_random_halos(n_halos):
    """Generate random positions for null test"""
    random_halos = []
    for i in range(n_halos):
        lon = np.random.uniform(0, 360)
        lat = np.random.uniform(-l_max, l_max)
        if abs(lat) > l_min:
            r200_deg = np.random.lognormal(np.log(1.0), 0.3)
            redshift = np.random.normal(0.05, 0.02)
            random_halos.append((lon, lat, r200_deg, max(0.01, redshift)))
    return random_halos

In [ ]:
manticore_data.keys()

In [ ]:
# ─── Bootstrap Methodology Validation Test (Final Enhanced Plot) ───

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from stacking_backend.utils import mpc_to_angular_degrees
# assume `pipeline` and your data dicts (manticore_data, eros_data, mcxc_data) are already in scope

# ─── CONFIGURATION ────────────────────────────────────────────────────

mass_bins = [
    (1e14, 2e14),
    (2e14, 3e14),
    (3e14, 4e14),
    (4e14, 5e14),
]
selected_mass_bin_index = 1

datasets = {
    'manticore': manticore_data,
    'erosita':  eros_data,
    'mcxc':     mcxc_data
}

inner_r200_factor = 1.2
outer_r200_factor = 6.0
patch_size_deg    = 20.0
npix              = 256
z_min, z_max      = 0.005, 0.06
b_min, b_max      = 10, 80
n_radial_bins     = 50
min_coverage      = 0.8

# ─── RUN ANALYSIS ─────────────────────────────────────────────────────

results_dict = {}
for ds_name, ds_data in datasets.items():
    results_dict[ds_name] = {}
    for m1, m2 in mass_bins:
        label = f"{m1:.1e}-{m2:.1e}"
        coords = []
        for l, b, z, m500, r500 in zip(
                ds_data['gal_l'], ds_data['gal_b'],
                ds_data['z'],    ds_data['m500'], ds_data['r500']
            ):
            if (
                z_min < z < z_max and
                b_min < abs(b) < b_max and
                m1 <= m500 <= m2
            ):
                coords.append([l, b,
                               mpc_to_angular_degrees(r500, z),
                               z])
        coords = np.array(coords)
        if coords.size == 0:
            continue

        res = pipeline.run_individual_r200_analysis_with_validation(
            coord_list=coords,
            inner_r200_factor=inner_r200_factor,
            outer_r200_factor=outer_r200_factor,
            patch_size_deg=patch_size_deg,
            npix=npix,
            min_coverage=min_coverage,
            n_radial_bins=n_radial_bins,
            run_null_tests=False,
            n_bootstrap=100,
            n_random=0
        )
        results_dict[ds_name][label] = res

# pick mass‐label for top row
m1, m2 = mass_bins[selected_mass_bin_index]
selected_label = f"{m1:.1e}-{m2:.1e}"

# ─── GLOBAL vmin/vmax FOR PATCHES ─────────────────────────────────────
vmin, vmax = -1e-6, 1e-6

# ─── MASS→COLOR MAPPING ────────────────────────────────────────────────
mass_centers = np.sqrt(np.array([x*y for x,y in mass_bins]))
cmap = plt.cm.Set1
norm = Normalize(vmin=mass_centers.min(), vmax=mass_centers.max())
colors = cmap(norm(mass_centers))

# ─── FIGURE SETUP ──────────────────────────────────────────────────────
with plt.style.context(get_mplstyle_path(mnras_style)):
    fig, axes = plt.subplots(
        2, 3, figsize=(6.5, 4),
        gridspec_kw={'height_ratios': [1, 1], 'hspace': 0.2, 'wspace': 0.1}
    )
    dataset_names = ['manticore', 'erosita', 'mcxc']
    dataset_labels = [r'$\texttt{Manticore-Local}$', 'eROSITA', 'MCXC']
    top_images = []
    
    # ── Top row: Stacked patches with text annotations ─────────────────────
    for col, (ds, ds_label) in enumerate(zip(dataset_names, dataset_labels)):
        ax = axes[0, col]
        res = results_dict[ds].get(selected_label, {})
        patch = res.get('stacked_patch')
        if patch is not None:
            extent = [-patch_size_deg/2, patch_size_deg/2,
                      -patch_size_deg/2, patch_size_deg/2]
            im = ax.imshow(patch, origin='lower', extent=extent,
                           cmap='RdBu_r', vmin=vmin, vmax=vmax)
            top_images.append(im)
    
            # center marker & apertures
            ax.plot(0, 0, 'k+', markersize=10)
            circ_in = plt.Circle((0, 0),
                                 res['r200_median'] * inner_r200_factor,
                                 fill=False, color='white', linewidth=2)
            circ_out = plt.Circle((0, 0),
                                  res['r200_median'] * outer_r200_factor,
                                  fill=False, color='white', linewidth=2, linestyle='--')
            ax.add_patch(circ_in)
            ax.add_patch(circ_out)
    
            # annotate mass range & sigma
            sigma = res.get('significance', np.nan)
            txt = f"{m1/1e14:.0f}–{m2/1e14:.0f} " + r"$\times 10^{14}$ M$_{\odot}$" + "\n" + r"$\sigma=$" + f"{sigma:.1f}"
            ax.text(0.95, 0.05, txt,
                    transform=ax.transAxes,
                    ha='right', va='bottom',
                    fontsize=8,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
            ax.set_xlim(-patch_size_deg/2, patch_size_deg/2)
            ax.set_ylim(-patch_size_deg/2, patch_size_deg/2)
            if col == 0:
                ax.set_ylabel(r'$\Delta$Dec [$\circ$]')
            ax.set_xlabel(r'$\Delta$RA [$\circ$]')
            ax.set_title(ds_label, pad=8)
            ax.grid(alpha=0.3)
            ax.set_aspect('equal')
        
            # turn off y-tick labels on columns 2 and 3 of the top row
            if col > 0:
                ax.tick_params(labelleft=False)
        
    # shared vertical colorbar for top row
    cbar_top = fig.colorbar(
        top_images[0], ax=axes[0, :],
        orientation='vertical', shrink=0.8, pad=0.02
    )
    cbar_top.set_label('Y [arbitrary units]')
    
    # ── Bottom row: Radial profiles with σ & N legends & mass colorbar ────
    for col, ds in enumerate(dataset_names):
        ax = axes[1, col]
        for idx, (m1, m2) in enumerate(mass_bins):
            label_key = f"{m1:.1e}-{m2:.1e}"
            res = results_dict[ds].get(label_key, {})
            r = res.get('profile_radii')
            y = res.get('profile_mean')
            e = res.get('profile_errors')
            nsrc = res.get('n_measurements', 0)
            if r is None or y is None:
                continue
    
            mask = np.isfinite(r) & np.isfinite(y)
            if e is not None:
                mask &= np.isfinite(e)
            if not mask.any():
                continue
    
            sigma = res.get('significance', np.nan)
            ax.errorbar(
                r[mask], y[mask],
                yerr=e[mask] if e is not None else None,
                fmt='o-', color=colors[idx],
                capsize=3, markersize=2, linewidth=1,
                label=r"$\sigma=$" + f"{sigma:.1f}, " + f"$N=$" + f"{nsrc}"
            )
    
        ax.axhline(0, color='black', linewidth=1, alpha=0.3)
        ax.grid(alpha=0.3)
        ax.set_xlim(-0.5, 3)
        ax.set_ylim(0, 1e-5)
        if col == 0:
            ax.set_ylabel('Y-parameter')
        else:
            ax.tick_params(labelleft=False)
        ax.set_xlabel(r'Radius [$\circ$]')
        ax.legend(loc='upper right', fontsize=7, frameon=False)
    
    # shared vertical colorbar for bottom row (mass → color)
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar_bot = fig.colorbar(
        sm, ax=axes[1, :],
        orientation='vertical', shrink=0.8, pad=0.02
    )
    cbar_bot.set_label(r'Mass [$10^{14}$ M$_{\odot}$]')
    cbar_bot.set_ticks(mass_centers)
    cbar_bot.set_ticklabels([f"{x/1e14:.0f}–{y/1e14:.0f}" for x, y in mass_bins])
    
    # ─── FINAL LAYOUT ───────────────────────────────────────────────────────
    plt.tight_layout(pad=0.1)
    plt.savefig("./plots/stacked_patches.pdf")
    plt.show()
